In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np

In [4]:
def create_scrollable_table(df, table_id, title):
    html = f'<h3>{title}</h3>'
    html += f'<div id="{table_id}" style="height:200px; overflow:auto;">'
    html += df.to_html()
    html += '</div>'
    return html

In [5]:
df = pd.read_csv("artifacts/data_ingestion/cars.csv")
df.head()

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150,Petrol,Manual,Grey,First,Corporate,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000,Diesel,Manual,White,Second,Individual,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000,Petrol,Manual,Maroon,First,Individual,1197 cc,79 bhp @ 6000 rpm,113 Nm @ 4000 rpm,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,Glanza G,799000,2019,37500,Petrol,Manual,Red,First,Individual,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000,Diesel,Manual,Grey,First,Individual,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,RWD,4735.0,1830.0,1795.0,7.0,55.0


In [6]:
df["TorquePower"] = [None if str(i) == "nan" else int(i.split("@")[0]) if len(i.split(" ")) == 1 else int(i.split(" ")[0]) for i in df["Max Torque"]]
df["TorquePowerRPM"] = [None if str(i) == "nan" else int(i.split("@")[1]) if len(i.split(" ")) == 1 else int(i.split(" ")[-2]) for i in df["Max Torque"]]
df["HorsePower"] = [None if str(i) == "nan" else int(i.split("@")[0]) if len(i.split(" ")) == 1 else int(i.split(" ")[0]) for i in df["Max Power"]]
df["HorsePowerRPM"] = [None if str(i) == "nan" else int(i.split("@")[1]) if len(i.split(" ")) == 1 else int(i.split(" ")[-2]) for i in df["Max Power"]]
df["Engine"] = [None if str(i) == "nan" else int(i.split(" ")[0]) for i in df["Engine"]]

In [7]:
df.drop(["Max Power","Model","Max Torque"], axis=1, inplace=True)

In [8]:
df.columns

Index(['Make', 'Price', 'Year', 'Kilometer', 'Fuel Type', 'Transmission',
       'Color', 'Owner', 'Seller Type', 'Engine', 'Drivetrain', 'Length',
       'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity',
       'TorquePower', 'TorquePowerRPM', 'HorsePower', 'HorsePowerRPM'],
      dtype='object')

In [9]:
df.shape

(2041, 20)

In [10]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2041 entries, 0 to 2040
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Make                2041 non-null   object 
 1   Price               2041 non-null   int64  
 2   Year                2041 non-null   int64  
 3   Kilometer           2041 non-null   int64  
 4   Fuel Type           2041 non-null   object 
 5   Transmission        2041 non-null   object 
 6   Color               2041 non-null   object 
 7   Owner               2041 non-null   object 
 8   Seller Type         2041 non-null   object 
 9   Engine              1961 non-null   float64
 10  Drivetrain          1907 non-null   object 
 11  Length              1977 non-null   float64
 12  Width               1977 non-null   float64
 13  Height              1977 non-null   float64
 14  Seating Capacity    1977 non-null   float64
 15  Fuel Tank Capacity  1928 non-null   float64
 16  Torque

In [12]:
df.duplicated().sum()

5

In [13]:
df.drop_duplicates(inplace=True)

In [14]:
df.duplicated().sum()

0

In [15]:
df.nunique()

Make                   33
Price                 618
Year                   22
Kilometer             838
Fuel Type               9
Transmission            2
Color                  17
Owner                   6
Seller Type             3
Engine                106
Drivetrain              3
Length                247
Width                 169
Height                195
Seating Capacity        6
Fuel Tank Capacity     55
TorquePower           135
TorquePowerRPM         56
HorsePower            164
HorsePowerRPM          41
dtype: int64

In [16]:
# Summary statistics for numerical features
numerical_features = df.select_dtypes(include=[np.number])
summary_stats = numerical_features.describe().T
html_numerical = create_scrollable_table(summary_stats, 'numerical_features', 'Summary statistics for numerical features')

display(HTML(html_numerical))

,count,mean,std,min,25%,50%,75%,max
Price,2036.0,1.709218e+06,2.426129e+06,49000.0,490000.0,840000.0,1935000.0,35000000.0
Year,2036.0,2.016447e+03,3.355979e+00,1988.0,2014.0,2017.0,2019.0,2022.0
Kilometer,2036.0,5.425077e+04,5.757660e+04,0.0,29000.0,50000.0,72000.0,2000000.0
Engine,1956.0,1.695297e+03,6.449651e+02,624.0,1198.0,1498.0,1995.0,6592.0
Length,1972.0,4.284056e+03,4.419874e+02,3099.0,3985.0,4370.0,4630.0,5569.0
Width,1972.0,1.768770e+03,1.354529e+02,1475.0,1695.0,1775.0,1835.0,2220.0
Height,1972.0,1.592532e+03,1.364336e+02,1165.0,1485.0,1548.0,1678.0,1995.0
Seating Capacity,1972.0,5.310852e+00,8.252877e-01,2.0,5.0,5.0,5.0,8.0
Fuel Tank Capacity,1924.0,5.209787e+01,1.513366e+01,15.0,42.0,50.0,60.0,105.0
TorquePower,1956.0,2.465440e+02,1.405883e+02,48.0,115.0,200.0,350.0,780.0


In [17]:
# Summary statistics for categorical features
categorical_features = df.select_dtypes(include=[object])
cat_summary_stats = categorical_features.describe().T
html_categorical = create_scrollable_table(cat_summary_stats, 'categorical_features', 'Summary statistics for categorical features')

display(HTML(html_categorical ))

,count,unique,top,freq
Make,2036,33,Maruti Suzuki,438
Fuel Type,2036,9,Diesel,1042
Transmission,2036,2,Manual,1117
Color,2036,17,White,793
Owner,2036,6,First,1601
Seller Type,2036,3,Individual,1974
Drivetrain,1902,3,FWD,1311


In [18]:
# Null values in the dataset
null_values = df.isnull().sum()
html_null_values = create_scrollable_table(null_values.to_frame(), 'null_values', 'Null values in the dataset')

# Percentage of missing values for each feature
missing_percentage = (df.isnull().sum() / len(df)) * 100
html_missing_percentage = create_scrollable_table(missing_percentage.to_frame(), 'missing_percentage', 'Percentage of missing values for each feature')

display(HTML(html_null_values + html_missing_percentage))

,0
Make,0
Price,0
Year,0
Kilometer,0
Fuel Type,0
Transmission,0
Color,0
Owner,0
Seller Type,0
Engine,80


In [19]:
num_features = df.select_dtypes(exclude="object").columns
cat_features = df.select_dtypes(include="object").columns

In [20]:
df[num_features] = df[num_features].fillna(df[num_features].mean())
df[cat_features] = df[cat_features].fillna(df[cat_features].mode().iloc[0])

In [21]:
# Null values in the dataset
null_values = df.isnull().sum()
html_null_values = create_scrollable_table(null_values.to_frame(), 'null_values', 'Null values in the dataset')

# Percentage of missing values for each feature
missing_percentage = (df.isnull().sum() / len(df)) * 100
html_missing_percentage = create_scrollable_table(missing_percentage.to_frame(), 'missing_percentage', 'Percentage of missing values for each feature')

display(HTML(html_null_values + html_missing_percentage))

,0
Make,0
Price,0
Year,0
Kilometer,0
Fuel Type,0
Transmission,0
Color,0
Owner,0
Seller Type,0
Engine,0


In [22]:
data = df

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2036 entries, 0 to 2040
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Make                2036 non-null   object 
 1   Price               2036 non-null   int64  
 2   Year                2036 non-null   int64  
 3   Kilometer           2036 non-null   int64  
 4   Fuel Type           2036 non-null   object 
 5   Transmission        2036 non-null   object 
 6   Color               2036 non-null   object 
 7   Owner               2036 non-null   object 
 8   Seller Type         2036 non-null   object 
 9   Engine              2036 non-null   float64
 10  Drivetrain          2036 non-null   object 
 11  Length              2036 non-null   float64
 12  Width               2036 non-null   float64
 13  Height              2036 non-null   float64
 14  Seating Capacity    2036 non-null   float64
 15  Fuel Tank Capacity  2036 non-null   float64
 16  TorquePower

In [24]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [25]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [27]:
import os
from mlProject import logger

In [28]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [29]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-08-08 14:58:04,402: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-08 14:58:04,404: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-08 14:58:04,406: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-08 14:58:04,407: INFO: common: created directory at: artifacts]
[2023-08-08 14:58:04,407: INFO: common: created directory at: artifacts/data_validation]
